In [1]:
#import need packages

import numpy as np
import random
import tensorflow as tf


In [2]:
# create train data
np.random.seed(3)
train_input_data = np.random.randn(1000, 8,4)
train_input_label = (np.absolute(train_input_data)).astype(int).max(axis = 1)
print(train_input_label.shape)
dev_input_data = np.random.randn(10,8,4)
dev_input_label = np.absolute(dev_input_data.copy()).astype(int).max(axis = 1)

(1000, 4)


In [4]:
print("train set input")
print(train_input_data[:10])
print("train set label")
print(train_input_label[:10])
print("dev set")
print(dev_input_data[:10])
print("dev set label")
print(dev_input_label[:10])

train set input
[[[  1.78862847e+00   4.36509851e-01   9.64974681e-02  -1.86349270e+00]
  [ -2.77388203e-01  -3.54758979e-01  -8.27414815e-02  -6.27000677e-01]
  [ -4.38181690e-02  -4.77218030e-01  -1.31386475e+00   8.84622380e-01]
  [  8.81318042e-01   1.70957306e+00   5.00336422e-02  -4.04677415e-01]
  [ -5.45359948e-01  -1.54647732e+00   9.82367434e-01  -1.10106763e+00]
  [ -1.18504653e+00  -2.05649899e-01   1.48614836e+00   2.36716267e-01]
  [ -1.02378514e+00  -7.12993200e-01   6.25244966e-01  -1.60513363e-01]
  [ -7.68836350e-01  -2.30030722e-01   7.45056266e-01   1.97611078e+00]]

 [[ -1.24412333e+00  -6.26416911e-01  -8.03766095e-01  -2.41908317e+00]
  [ -9.23792022e-01  -1.02387576e+00   1.12397796e+00  -1.31914233e-01]
  [ -1.62328545e+00   6.46675452e-01  -3.56270759e-01  -1.74314104e+00]
  [ -5.96649642e-01  -5.88594380e-01  -8.73882298e-01   2.97138154e-02]
  [ -2.24825777e+00  -2.67761865e-01   1.01318344e+00   8.52797841e-01]
  [  1.10818750e+00   1.11939066e+00   1.48754

# Building NN Model

In [15]:
def model(w, b, kind, width, height):
    #1 layer hidden unit
    x = tf.placeholder(tf.float32, [None, width * height])
    x_image = tf.reshape(x, shape=[-1, height, width, 1])
    y_ = tf.placeholder(tf.float32, [None, kind])
    keep_prob = tf.placeholder(tf.float32)
    w_alpha = 0.00001
    b_alpha = 0.0001
    layers_dims = [1, 32]
    L = len(layers_dims)
    
    input_layer_data = tf.reshape(x_image, [-1, height, width, 1])
    new_width, new_height = width, height
    for l in range(1,L):
        input_layer_data = tf.reshape(input_layer_data, [-1, height, width, 1])
        w = tf.Variable(w_alpha*tf.random_normal([3, 3, layers_dims[l-1], layers_dims[l]])) 
        b = tf.Variable(b_alpha*tf.random_normal([layers_dims[l]]))
        conv = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(input_layer_data, w, strides=[1, 1, 1, 1], padding='SAME'), b))
        conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv = tf.nn.dropout(conv, keep_prob)
        input_layer_data = conv
        new_width = int(new_width / 2)
        new_height = int(new_height / 2) 

        
    w_d = tf.Variable(w_alpha*tf.random_normal([int(new_width * new_height * layers_dims[l]), 256]))
    b_d = tf.Variable(b_alpha*tf.random_normal([256]))
    dense = tf.reshape(conv, [-1, w_d.get_shape().as_list()[0]])
    dense = tf.nn.relu(tf.add(tf.matmul(dense, w_d), b_d))
    dense = tf.nn.dropout(dense, keep_prob)

    w_out1 = tf.Variable(w_alpha*tf.random_normal([256, 1]))
    b_out1 = tf.Variable(b_alpha*tf.random_normal([1]))
    w_out2 = tf.Variable(w_alpha*tf.random_normal([256, 1]))
    b_out2 = tf.Variable(b_alpha*tf.random_normal([1]))
    w_out3 = tf.Variable(w_alpha*tf.random_normal([256, 1]))
    b_out3 = tf.Variable(b_alpha*tf.random_normal([1]))
    w_out4 = tf.Variable(w_alpha*tf.random_normal([256, 1]))
    b_out4 = tf.Variable(b_alpha*tf.random_normal([1]))

    y_conv1 = tf.add(tf.matmul(dense, w_out1), b_out1)    
    y_conv2 = tf.add(tf.matmul(dense, w_out2), b_out2)    
    y_conv3 = tf.add(tf.matmul(dense, w_out3), b_out3)    
    y_conv4 = tf.add(tf.matmul(dense, w_out4), b_out4)
    out = tf.concat([y_conv1, y_conv2, y_conv3, y_conv3],1, name="out")
    return x, keep_prob, y_, out


In [ ]:
width = train_input_data.shape[2]
height = train_input_data.shape[1]
input_data = train_input_data.reshape((train_input_data.shape[0], train_input_data.shape[1]*train_input_data.shape[2]))
x, keep_prob, y_, out = model(0.001, 0.001, 4, width, height)

#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = y_group, labels=y_))
loss = tf.reduce_mean(tf.pow((out - y_),2))
#loss = tf.reduce_mean(tf.pow(out - y_))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(loss)
#get closest int
out_rint = tf.cast(tf.rint(out), tf.int64)
correct_prediction = tf.equal(tf.cast(y_, tf.int64), out_rint)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("start")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100000) :
        _, loss_ = sess.run([optimizer, loss], feed_dict={ x: input_data, y_: train_input_label, keep_prob: 0.5}) 

        if(i % 300 == 0) :
            accuracy_result = sess.run(accuracy , feed_dict={ x: input_data , y_: train_input_label , keep_prob: 1}) 
            print("step - "+ str(i), " train loss is ", loss_)
            print("train accuracy", accuracy_result)



start
step - 0  train loss is  1.95714
train accuracy 0.05225
step - 300  train loss is  0.332492
train accuracy 0.65525
step - 600  train loss is  0.313369
train accuracy 0.67625


In [20]:
with tf.Session() as sess:
    m = sess.run(tf.truncated_normal((5,10), stddev = 0.1) )
    print(m)
    col_max = sess.run(tf.argmax(m, 0) )  #当axis=0时返回每一列的最大值的位置索引
    print(col_max)

[[-0.04593372  0.02352516 -0.11056554  0.01708604 -0.15671374  0.12785572
   0.02952961  0.0468499  -0.02154115  0.00945626]
 [-0.04678419  0.14627931 -0.07190793 -0.00212238  0.07568391  0.04104963
  -0.11839457  0.11952905  0.08296504  0.03865674]
 [ 0.1440687  -0.04192343  0.16179623  0.08320116  0.09332499  0.08333122
  -0.10055806  0.14446025 -0.09452751  0.03892508]
 [ 0.0069762  -0.13830023  0.04476939 -0.1452758  -0.04128321  0.05971946
  -0.02719602  0.01412355  0.0238658   0.11327913]
 [ 0.02058141  0.07456981  0.08213323  0.00642623  0.19259399 -0.02323579
   0.19925457  0.04350021 -0.10422335 -0.0143043 ]]
[2 1 2 2 4 0 4 2 1 3]
